# RapidFire AI - Colab Installation Test

This notebook tests the Google Colab support implementation.

**Branch**: `feat/enable-colab`

## 1. Check Environment

In [ ]:
# Check GPU
!nvidia-smi

# Check Python version
import sys
print(f"\nPython version: {sys.version}")

# Check if in Colab
try:
    import google.colab
    print("✅ Running in Google Colab")
    IN_COLAB = True
except ImportError:
    print("❌ NOT running in Google Colab")
    IN_COLAB = False

## 2. Install RapidFire from GitHub Branch

Installing from `feat/enable-colab` branch...

In [ ]:
# Install from GitHub branch
!pip install -q git+https://github.com/RapidFireAI/rapidfireai.git@feat/enable-colab

print("\n" + "="*60)
print("Installation complete!")
print("="*60)

## 3. Verify Installation

In [ ]:
# Check version
!rapidfireai --version

# Check if command exists
import subprocess
result = subprocess.run(['which', 'rapidfireai'], capture_output=True, text=True)
print(f"\nRapidFire CLI location: {result.stdout.strip()}")

# Try importing modules
print("\nImporting modules...")
try:
    from rapidfireai.utils.colab_helper import (
        is_colab,
        get_notebook_environment,
        expose_port_native,
        setup_cloudflare_tunnel,
        expose_rapidfire_services
    )
    print("✅ colab_helper module imported successfully")
except ImportError as e:
    print(f"❌ Failed to import colab_helper: {e}")

try:
    from rapidfireai import Experiment
    print("✅ Experiment class imported successfully")
except ImportError as e:
    print(f"❌ Failed to import Experiment: {e}")

## 4. Test Environment Detection

In [ ]:
from rapidfireai.utils.colab_helper import is_colab, is_kaggle, get_notebook_environment

print("Environment Detection:")
print(f"  is_colab(): {is_colab()}")
print(f"  is_kaggle(): {is_kaggle()}")
print(f"  get_notebook_environment(): {get_notebook_environment()}")

if is_colab():
    print("\n✅ Colab environment detected correctly!")
else:
    print("\n⚠️ Colab environment NOT detected (are you in Colab?)")

## 5. Test Port Forwarding Functions

Test that the port forwarding functions exist and can be called.

In [ ]:
from rapidfireai.utils.colab_helper import (
    expose_port_native,
    expose_port_iframe,
    setup_cloudflare_tunnel,
    setup_ngrok_tunnel
)

print("Testing function availability:")
print(f"  expose_port_native: {callable(expose_port_native)}")
print(f"  expose_port_iframe: {callable(expose_port_iframe)}")
print(f"  setup_cloudflare_tunnel: {callable(setup_cloudflare_tunnel)}")
print(f"  setup_ngrok_tunnel: {callable(setup_ngrok_tunnel)}")

print("\n✅ All port forwarding functions are available")

## 6. Test CLI --colab Flag

In [ ]:
# Test help to see if --colab flag exists
!rapidfireai --help

## 7. Test Init Command

In [ ]:
# Test init (installs dependencies and copies tutorials)
!rapidfireai init

# Check if tutorial notebooks were copied
import os
tutorial_path = os.path.join(".", "tutorial_notebooks")
if os.path.exists(tutorial_path):
    notebooks = [f for f in os.listdir(tutorial_path) if f.endswith('.ipynb')]
    print(f"\n✅ Found {len(notebooks)} tutorial notebooks:")
    for nb in notebooks[:5]:  # Show first 5
        print(f"  - {nb}")
    if len(notebooks) > 5:
        print(f"  ... and {len(notebooks) - 5} more")
else:
    print(f"\n⚠️ Tutorial notebooks not found at {tutorial_path}")

## 8. Check Dependency Conflicts

In [ ]:
# Check for dependency conflicts
import subprocess
import sys

print("Checking for dependency conflicts...\n")
result = subprocess.run(
    [sys.executable, '-m', 'pip', 'check'],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ No dependency conflicts found!")
else:
    print("⚠️ Some dependency conflicts exist:")
    print(result.stdout)
    print("\nNote: Minor version conflicts with Colab packages are expected and usually safe.")

## 9. Check Installed Package Versions

In [ ]:
import subprocess
import sys

packages = [
    'rapidfireai',
    'pandas',
    'ipython',
    'requests',
    'torch',
    'transformers',
    'mlflow',
    'flask',
    'gunicorn'
]

print("Key package versions:\n")
for package in packages:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'show', package],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        version_line = [line for line in result.stdout.split('\n') if line.startswith('Version:')]
        if version_line:
            version = version_line[0].split(':')[1].strip()
            print(f"  {package}: {version}")
    else:
        print(f"  {package}: NOT INSTALLED")

## 10. Test Service Startup (DRY RUN)

**Note**: This will actually start the services. You can skip this cell if you just want to test installation.

In [ ]:
# Just test that the command is recognized
# We won't actually start services here (that's in the next section)

import subprocess
import sys

# Test that the command parses correctly
result = subprocess.run(
    [sys.executable, '-m', 'rapidfireai.cli', 'start', '--colab', '--help'],
    capture_output=True,
    text=True,
    timeout=5
)

if '--colab' in result.stdout or '--colab' in result.stderr:
    print("✅ CLI recognizes --colab flag")
else:
    print("⚠️ CLI might not recognize --colab flag")
    print("Output:", result.stdout)
    print("Error:", result.stderr)

## ✅ Test Summary

If all cells above completed without errors, the installation is successful!

### What was tested:
- ✅ Package installation from GitHub branch
- ✅ CLI command availability
- ✅ Module imports (colab_helper, Experiment)
- ✅ Environment detection (is_colab)
- ✅ Port forwarding function availability
- ✅ CLI --colab flag recognition
- ✅ Init command (dependencies and tutorials)
- ✅ Dependency compatibility

### Next Steps:

1. **Test actual service startup** (see next section)
2. **Test port forwarding** with live services
3. **Run a simple experiment** to verify end-to-end functionality

---

## 11. Start RapidFire Services (OPTIONAL)

**⚠️ Warning**: This will start services and keep running. Only run this if you want to test the actual service startup.

Uncomment and run the cell below to start services:

In [ ]:
# Uncomment to test actual service startup:

# !rapidfireai start --colab

# Or test with Cloudflare:
# !rapidfireai start --colab --tunnel cloudflare

print("Service startup test skipped (uncomment to run)")

## 12. Test Python API (Alternative to CLI)

In [ ]:
# Test the Python API for exposing services
from rapidfireai.utils.colab_helper import expose_rapidfire_services

print("Testing expose_rapidfire_services() function...\n")

# We won't actually start services, just test the function signature
import inspect
sig = inspect.signature(expose_rapidfire_services)
print(f"Function signature: {sig}")

params = list(sig.parameters.keys())
print(f"\nParameters: {params}")

if 'method' in params:
    print("\n✅ Python API is available and has correct signature")
else:
    print("\n⚠️ Function signature might be incorrect")

---

## 📝 Testing Notes

### Known Issues to Watch For:

1. **Dependency Conflicts**:
   - Minor version conflicts with Colab's pre-installed packages are expected
   - Look for: pandas, ipython, requests versions
   - These are usually safe if they're close versions

2. **Python Version**:
   - Colab uses Python 3.10.x
   - RapidFire now supports Python >= 3.10
   - Should work without issues

3. **Port Forwarding**:
   - Native Colab forwarding only works in actual Colab environment
   - Won't work in local Jupyter or other environments

### What to Report:

If you encounter issues, please report:
- Which cell failed
- Full error message
- Output of `!rapidfireai --version`
- Output of dependency check (cell 8)

---

**Installation test complete!** 🎉